In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import syft as sy
from syft.core.node.common.action.get_object_action import GetObjectAction
from syft.core.node.common.action.function_or_constructor_action import RunFunctionOrConstructorAction
from syft.core.node.common.action.common import Action
from syft.proto.core.node.common.plan.plan_pb2 import Plan as Plan_PB

from typing import List
from syft.core.common.uid import UID
from syft.core.io.address import Address
from syft.core.common.serde.deserialize import _deserialize
from syft.core.common.object import Serializable
from syft.proto.core.node.common.action.action_pb2 import Action as Action_PB
from syft.core.node.common.plan.plan import Plan
import torch as th
import sys

# Plans

## Serialization

In [3]:
a1 = GetObjectAction(id_at_location=UID(), address=Address(), reply_to=Address(), msg_id=UID())
a2 = RunFunctionOrConstructorAction(path="torch.Tensor.add",args=tuple(),kwargs={},id_at_location=UID(),
                                    address=Address(),msg_id=UID())

In [4]:
plan = Plan([a1,a2])

In [5]:
blob = plan.serialize()
plan_reconstructed = sy.deserialize(blob=blob)

In [6]:
# %debug

In [7]:
assert isinstance(plan_reconstructed, Plan)
assert all(isinstance(a, Action) for a in plan_reconstructed.actions)

## Execution

In [8]:
alice = sy.VirtualMachine(name="alice")
alice_client = alice.get_client()

In [9]:
t = th.tensor([1,2,3])
tensor_pointer = t.send(alice_client)

In [10]:
action = GetObjectAction(id_at_location=tensor_pointer.id_at_location, address=Address(),
                         reply_to=Address(), msg_id=UID())
plan = Plan([action])

In [11]:
plan_pointer = plan.send(alice_client)
result = plan_pointer.execute()

In [12]:
x = result.get()

In [13]:
x

In [14]:
result

In [15]:
plan_pointer

In [16]:
alice_client

<VirtualMachineClient: alice Client>

[2021-02-17T14:32:13.393320+0100][CRITICAL][logger]][4478] <class 'syft.core.store.store_memory.MemoryStore'> __getitem__ error <UID: 7d5986955fd948eb930fefd7af1e8b0a> <UID: 7d5986955fd948eb930fefd7af1e8b0a>
[2021-02-17T14:32:13.393660+0100][CRITICAL][logger]][4478] <UID: 7d5986955fd948eb930fefd7af1e8b0a>
[2021-02-17T14:32:13.394340+0100][CRITICAL][logger]][4478] Unable to Get Object with ID <UID: 7d5986955fd948eb930fefd7af1e8b0a> from store. Possible dangling Pointer. <UID: 7d5986955fd948eb930fefd7af1e8b0a>
[2021-02-17T14:32:13.394819+0100][CRITICAL][logger]][4478] Unable to Get Object with ID <UID: 7d5986955fd948eb930fefd7af1e8b0a> from store. Possible dangling Pointer. <UID: 7d5986955fd948eb930fefd7af1e8b0a>


Exception: Unable to Get Object with ID <UID: 7d5986955fd948eb930fefd7af1e8b0a> from store. Possible dangling Pointer. <UID: 7d5986955fd948eb930fefd7af1e8b0a>

In [29]:
%debug

> /Users/koen/workspace/PySyft/src/syft/core/node/common/plan/plan.py(24)execute()
     22     def execute(self):
     23         for a in self.actions:
---> 24             a.execute_action()
     25 
     26     @staticmethod

ipdb> u
> /Users/koen/workspace/PySyft/src/syft/core/node/common/action/run_class_method_action.py(161)execute_action()
    159                     )
    160             else:
--> 161                 result = method(resolved_self.data, *upcasted_args, **upcasted_kwargs)
    162 
    163         if lib.python.primitive_factory.isprimitive(value=result):

ipdb> node
VirtualMachine: alice: <UID: f5e39ef94ae64619b7ee90f23b7e93ca>
ipdb> u
> /Users/koen/workspace/PySyft/src/syft/core/node/common/service/obj_action_service.py(27)process()
     25         verify_key: Optional[VerifyKey] = None,
     26     ) -> None:
---> 27         msg.execute_action(node=node, verify_key=verify_key)
     28 
     29     @staticmethod

ipdb> ll
     21     @staticmethod
     22     def